In [1]:
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA
from Crypto.Signature import PKCS1_v1_5
from datetime import datetime as DT
import hmac
from Crypto.Cipher import AES
from merklelib import MerkleTree
import warnings
import random
warnings.filterwarnings("ignore")

In [2]:
invalidTxns=[]
flag=0
miner=[]

In [3]:
class device:
    deviceCount = 0
    
    def __init__(self):
        self.id = device.deviceCount
        self.sk,self.pk= self.getKeys()
        self.balance=100
        
        self.idProof = {}
        self.idProof['public_KEY'] = self.pk.hex()
        self.idProof['block_no'] = None
        self.idProof['block_hash'] = None
        self.idProof['difficulty'] = None
        self.idProof['txn_Index'] = None
        self.idProof['Node_ID'] = None
        self.idProof['okToSend'] = None
        self.idProof['okToRecieve'] = None
        self.confi=0.5
        self.resi=100
        self.txncount=0
        self.D=0.01
        
        data=str(self.id)+self.pk.hex()
        self.address=SHA.new(data.encode()).hexdigest()
        device.deviceCount += 1
    
    def getKeys(self):
        self.keypair=RSA.generate(1024)
        sk = self.keypair.exportKey('DER')
        pk = self.keypair.publickey().exportKey('DER')
        return sk,pk
    
    def getDeviceDetails(self):
        print("deviceID: "+str(self.id),"publicKEY:",self.pk.hex(),"deviceADDRESS:",self.address,sep="\n",end="\n\n")
        
    @classmethod
    def transact(self, S,R,amt):
        
        if amt < dList[S.id].balance:
            dList[S.id].balance-=amt
            dList[R.id].balance+=amt
            Txn= S.address+R.address+str(amt)
            sign = device.signTxn(Txn,S.sk)
            verified = device.verifyTxn(Txn, sign, S.pk)
            if(verified):
                print("Transaction is verified")
                if dList[S.id].resi>0:
                    dList[S.id].resi-=(random.randint(1,2)/100)
                    dList[S.id].txncount+=1
                    return [{"Sender":S,"Receiver":R,"Amount":amt}]
                else:
                    dList[S.id].resi=0
                return []
            else:
                invalidTxns.append({"Sender":dList[S],"Receiver":dList[R],"Amount":amt,"status":"TxnNotVerified"})
                print("Fraudulent transaction")
                return []
        else:
            invalidTxns.append({"Sender":S.id,"Receiver":R.id,"Amount":amt,"status":"TxnNotVerified"})
            print("Sybil node")
            return []
            
    @classmethod
    def dataTransfer(self,S,R,msg,dtKEY):
        sF,rF=device.checkID(S,R)
        if(not sF):
            print("TODO sF triggered")
        if(not rF):
            print("TODO rF triggered")
        else:
            dList[S.id].resi-=(random.randint(1,2)/100)
            pad=16-len(msg)%16
            msg=msg.zfill(pad+len(msg))
            secret=AES.new(dtKEY)
            cipherTxt=secret.encrypt(msg)
            signature=device.signTxn(cipherTxt,S.sk)
            dList[S.id].txncount+=1
            return [S,R,cipherTxt,signature]
        return None
        
    @classmethod
    def checkID(self, S, R):
        SID=S.idProof
        RID=R.idProof
        sF,rF=True,True    #F-> flag
        for i in SID.values():
            if(i is None):
                sF=False
                invalidTxns.append({"Sender":S,"Receiver":R,"Amount":amt,"status":"SenderIDNotVerified"})
                print("Sender ID cannot be verified")
                break
        for i in RID.values():
            if(i is None):
                rF=False
                invalidTxns.append({"Sender":S,"Receiver":R,"Amount":amt,"status":"ReceiverNotVerified"})
                print("Receiver ID cannot be verified")
                break
        return (sF,rF)
        
    
    @classmethod
    def signTxn(self,txn, sk):
        txn=str(txn)
        RSA_key=RSA.importKey(sk)
        author=PKCS1_v1_5.new(RSA_key)
        msg=SHA.new(txn.encode())
        signature = author.sign(msg)
        return signature
    
    @classmethod
    def verifyTxn(self, txn, sign, pk):
        txn=str(txn)
        msg=SHA.new(txn.encode())
        RSA_key=RSA.importKey(pk)
        verifier = PKCS1_v1_5.new(RSA_key)
        verified=verifier.verify(msg, sign)
        return verified
    
    def __del__(self):
        print("Device:",self.id,"deleted")
        device.deviceCount-=1

In [4]:
class block:
    blockCount=0
    def __init__(self,pH,txnList,PoW,mR):
        block.blockCount+=1
        self.blockNo=block.blockCount
        self.prevHash=pH
        self.transactions=txnList
        self.timestamp=DT.timestamp(DT.now())
        self.hash=PoW['Hash']
        self.nonce=PoW['nonce']
        self.mRoot=mR
    
    def getBlockDetails(self):
        print("Block#:",self.blockNo)
        print("previous HASH:",self.prevHash)
        print("current HASH:",self.hash)
        print()
        
        print("Transactions:",len(self.transactions),"     Time:",DT.fromtimestamp(self.timestamp))
        for T in self.transactions:
            print("Sender:",T['Sender'].id, end=",  ")
            print("Receiver:",T['Receiver'].id, end=",  ")
            print("Amount:",T['Amount'])
        
        print("Merkley Root:",self.mRoot)
        print("\nNONCE:",self.nonce)

In [5]:
def HF(data):
    data=data.hex()
    return SHA.new(data.encode()).hexdigest()

In [6]:
class blockChain:
    chain=[]
    allTxns={}
    Tcount=1
    def __init__(self,H):
        pH='0'
        PoW={}
        txn=[{"Sender":H,"Receiver":H,"Amount":0}]
        PoW['Hash'],PoW['nonce']=self.PoW(pH,txn)
        mR=SHA.new(str(txn).encode()).hexdigest()
        txID = str(1)
        diff=3
        H.idProof['block_no'] = len(blockChain.chain)
        H.idProof['block_hash'] = PoW['Hash']
        H.idProof['difficulty'] = diff
        H.idProof['txn_Index'] = txID
        pk=H.pk
        key=hmac.HMAC(pk)
        H.idProof['Node_ID'] = hmac.HMAC(key.digest(),txID.encode()).hexdigest()
        H.idProof['okToSend'] = '0'
        H.idProof['okToRecieve'] = '0'
        
        BLOC=block(pH,txn,PoW,mR)
        blockChain.chain.append(BLOC)
        
        
    def newBlock(self,txn):
        if(txn==[]):
            return
        
        Tno="Txn-"+str(blockChain.Tcount)
        blockChain.allTxns[Tno]=txn[0]
        blockChain.Tcount+=1
        pH=blockChain.chain[-1].hash
        PoW={}
        PoW['Hash'],PoW['nonce']=self.PoW(pH,txn)
        mT=MerkleTree(txn, HF)
        mR=mT.merkle_root
        #ID PROOF GENERATION
        S = txn[0]['Sender']
        txID = str(1)
        diff=3
        S.idProof['block_no'] = len(blockChain.chain)
        S.idProof['block_hash'] = PoW['Hash']
        S.idProof['difficulty'] = diff
        S.idProof['txn_Index'] = txID
        pk=S.pk
        key=hmac.HMAC(pk)
        S.idProof['Node_ID'] = hmac.HMAC(key.digest(),txID.encode()).hexdigest()
        S.idProof['okToSend'] = '0'
        S.idProof['okToRecieve'] = '0'
        
        BLOC=block(pH,txn,PoW,mR)
        miner.append(self.PoC())
        blockChain.chain.append(BLOC)
        
    def addNewBlock(self,txnList):
        if(txnList==[]):
            return
        
        for T in txnList:
            Tno="Txn-"+str(blockChain.Tcount)
            blockChain.allTxns[Tno]=T
            blockChain.Tcount+=1
            
        pH=blockChain.chain[-1].hash
        PoW={}
        PoW['Hash'],PoW['nonce']=self.PoW(pH,txnList)        
        mT = MerkleTree(txnList, HF)
        mR=mT.merkle_root
        BLOC=block(pH,txnList,PoW,mR)
        miner.append(self.PoC())
        blockChain.chain.append(BLOC)
    
    def PoC(self):
        maxr=0
        maxt=0
        for i in range(N):
            if dList[i].resi>=dList[maxr].resi:
                maxr=i
            if dList[i].txncount>=dList[maxt].txncount:
                maxt=i
        print(maxr)
        print(maxt)
        for i in range(N):
            dList[i].confi= (dList[i].confi*(1-dList[i].D))+ ((dList[i].txncount/dList[maxt].txncount)*(dList[i].resi/dList[maxr].resi))
            if i>0:
                if dList[i].confi > dList[i-1].confi:
                    maxco=dList[i].confi
        for i in range(N):
            dList[i].confi=dList[i].confi/maxco
            if(dList[i].confi==1):
                t=i
        for i in range(N):
            dList[i].txncount=0
        return t
        
    
    def PoW(self,pH,txn,diff=2):
        nonce=0
        data=pH+str(txn)+str(nonce)
        nH=SHA.new(data.encode()).hexdigest()
        while nH[:diff] != '0'*diff:
            nonce += 1
            data=pH+str(txn)+str(nonce)
            nH=SHA.new(data.encode()).hexdigest()
        return nH,nonce
    
    def getChainDetails(self):
        for D in blockChain.chain:
            D.getBlockDetails()
            print("-------------------------------------------")

In [7]:
dtKEY="sixteen--SIXTEEN"
N=int(input("Enter no.of devices:"))
dList=[]


for i in range(N):
    dList.append(device())

show=input("press \'Y' to show device details, any other key to skip:")
if(show in ['Y','y']):
    for i in dList:
        i.getDeviceDetails()

myChain=blockChain(dList[0])
for i in range(1,N):
    Txn = input("Sender#:   Receiver#:   Amount:").strip().split()
    S,R,amt = [int(x) for x in Txn]
    if(S>=len(dList)):
        invalidTxns.append({"Sender":dList[S],"Receiver":dList[R],"Amount":amt,"status":"SenderNotFound"})
        print("Sender not found")
    elif(S==0):
        invalidTxns.append({"Sender":dList[S],"Receiver":dList[R],"Amount":amt,"status":"inValidSender"})
        print("Invalid Sender")
    elif(R != 0):
        invalidTxns.append({"Sender":dList[S],"Receiver":dList[R],"Amount":amt,"status":"inValidReceiver"})
        print("Invalid Receiver")
    else:
        oneTxnList=device.transact(dList[S],dList[R],amt)
        myChain.newBlock(oneTxnList)

Enter no.of devices:3
press 'Y' to show device details, any other key to skip:y
deviceID: 0
publicKEY:
30819f300d06092a864886f70d010101050003818d0030818902818100b9ed705df8b0c535379e954413d2cce084fe3b7f9f38f67d7481bd28b79104bda36c8ffd124735971c286da8665255f08d1ee8c803915f430721b30cd156477e6809ce853b79e54cd8846d41fd1e17253d9e4aae0a2965dd76466cd60b15299a46129b6bfa520c2df5a031a30edc182665627569c7054273053415bf404057650203010001
deviceADDRESS:
5e0e23183b5575cb20fa13fec43d8e660d45777c

deviceID: 1
publicKEY:
30819f300d06092a864886f70d010101050003818d0030818902818100cf8fe68e29cf9b6735824a9779998fea3a578e2055a4d4e901adb3fab45febfeabec53f74708928039169dc834fab8d011c0fcc40b669611ef6249efb599ec543feb8ee0a73b8be1a4b6fa8d68de9b986b931f94149b7200e1b2f135cbd2d41799d4edb029b2cefdd3aa0b75155d75e4201ecaeda21c7eab0dda5d44af47fe150203010001
deviceADDRESS:
06d247ceb957a30e7bc014fc2b9b5d13e6193d16

deviceID: 2
publicKEY:
30819f300d06092a864886f70d010101050003818d0030818902818100f6589e90258ba871032e3654320b0

In [8]:
invalidTxns

[]

## Displaying each block

In [9]:
myChain.getChainDetails()

Block#: 1
previous HASH: 0
current HASH: 00712cd6cf4ebc14c9ba7fbb4963d675d224a856

Transactions: 1      Time: 2020-02-04 14:08:07.859978
Sender: 0,  Receiver: 0,  Amount: 0
Merkley Root: ddbaaf2a58ea4e923d156dc70b18614095399e41

NONCE: 256
-------------------------------------------
Block#: 2
previous HASH: 00712cd6cf4ebc14c9ba7fbb4963d675d224a856
current HASH: 00e35824f78df09583e13c1ddcf6e47175c445de

Transactions: 1      Time: 2020-02-04 14:09:02.427059
Sender: 1,  Receiver: 0,  Amount: 0
Merkley Root: 68f958e693017513ceb7f6af04d089d14174517c

NONCE: 385
-------------------------------------------
Block#: 3
previous HASH: 00e35824f78df09583e13c1ddcf6e47175c445de
current HASH: 0073c95fda71ba7e2d7ed1f39745b6b3cbc12315

Transactions: 1      Time: 2020-02-04 14:09:14.280881
Sender: 2,  Receiver: 0,  Amount: 0
Merkley Root: 11608d6f51cf2ec61d2bfaeccba50314f4bbea27

NONCE: 530
-------------------------------------------


In [10]:
for k,v in myChain.allTxns.items():
    print(k,": Sender:",v["Sender"].address, "   Receiver:",v["Receiver"].address,   "   Amount:",v['Amount'])        

Txn-1 : Sender: 06d247ceb957a30e7bc014fc2b9b5d13e6193d16    Receiver: 5e0e23183b5575cb20fa13fec43d8e660d45777c    Amount: 0
Txn-2 : Sender: 8ff34cc3e9c102d807333459a7f49731dd26a94c    Receiver: 5e0e23183b5575cb20fa13fec43d8e660d45777c    Amount: 0


### Displaying Identity Set

In [11]:
for D in dList:
    print("Device:",D.id)
    for k,v in D.idProof.items():
        print(k,":",v)
    print()

Device: 0
public_KEY : 30819f300d06092a864886f70d010101050003818d0030818902818100b9ed705df8b0c535379e954413d2cce084fe3b7f9f38f67d7481bd28b79104bda36c8ffd124735971c286da8665255f08d1ee8c803915f430721b30cd156477e6809ce853b79e54cd8846d41fd1e17253d9e4aae0a2965dd76466cd60b15299a46129b6bfa520c2df5a031a30edc182665627569c7054273053415bf404057650203010001
block_no : 0
block_hash : 00712cd6cf4ebc14c9ba7fbb4963d675d224a856
difficulty : 3
txn_Index : 1
Node_ID : 6a025c75ce39aef27f8ecae43cf1d9bd
okToSend : 0
okToRecieve : 0

Device: 1
public_KEY : 30819f300d06092a864886f70d010101050003818d0030818902818100cf8fe68e29cf9b6735824a9779998fea3a578e2055a4d4e901adb3fab45febfeabec53f74708928039169dc834fab8d011c0fcc40b669611ef6249efb599ec543feb8ee0a73b8be1a4b6fa8d68de9b986b931f94149b7200e1b2f135cbd2d41799d4edb029b2cefdd3aa0b75155d75e4201ecaeda21c7eab0dda5d44af47fe150203010001
block_no : 1
block_hash : 00e35824f78df09583e13c1ddcf6e47175c445de
difficulty : 3
txn_Index : 1
Node_ID : bd0f328f88845959af07e0cc8298e

## Data Transfer

In [18]:
txnList=[]
while(1):
    cmd=input("Press N to send new data, any other key to stop sending:")
    if(cmd in ['N','n']):

        S=int(input("Enter Sender's id:"))
        if(S>=len(dList) or S<0):
            invalidTxns.append({"Sender":dList[S],"Receiver":dList[R],"Amount":amt,"status":"inValidSender"})
            print("invalid sender")
            continue
        R=int(input("Enter Receiver's id:"))
        if(R>=len(dList) or R<0):
            invalidTxns.append({"Sender":dList[S],"Receiver":dList[R],"Amount":amt,"status":"inValidReceiver"})
            print("invalid Receiver")
            continue
        
        msg=input("Enter data to be sent:")
        
        data=device.dataTransfer(dList[S],dList[R],msg,dtKEY)
        if(not bool(data)):
            continue
        verified=device.verifyTxn(data[2],data[3],data[0].pk)
        if(verified):
            txnList.append({"Sender":data[0],"Receiver":data[1],"Amount":data[2].hex()})
#             print("Sender Address:  ",data[0].address)
#             print("Receiver Address:",data[1].address)
#             print("Encrypted msg:   ",data[2].hex())
            print("SUCCESSFUL")
            print()
        if(len(txnList)==3):
            myChain.addNewBlock(txnList)
            txnList=[]
    else:
        if(bool(txnList)):
            myChain.addNewBlock(txnList)
            txnList=[]
        break

Press N to send new data, any other key to stop sending:n
Enter Sender's id:1
Enter Receiver's id:2
Enter data to be sent:m1
SUCCESSFUL

Press N to send new data, any other key to stop sending:n
Enter Sender's id:2
Enter Receiver's id:1
Enter data to be sent:m2
SUCCESSFUL

Press N to send new data, any other key to stop sending:1
0
2


In [19]:
myChain.getChainDetails()

Block#: 1
previous HASH: 0
current HASH: 00712cd6cf4ebc14c9ba7fbb4963d675d224a856

Transactions: 1      Time: 2020-02-04 14:08:07.859978
Sender: 0,  Receiver: 0,  Amount: 0
Merkley Root: ddbaaf2a58ea4e923d156dc70b18614095399e41

NONCE: 256
-------------------------------------------
Block#: 2
previous HASH: 00712cd6cf4ebc14c9ba7fbb4963d675d224a856
current HASH: 00e35824f78df09583e13c1ddcf6e47175c445de

Transactions: 1      Time: 2020-02-04 14:09:02.427059
Sender: 1,  Receiver: 0,  Amount: 0
Merkley Root: 68f958e693017513ceb7f6af04d089d14174517c

NONCE: 385
-------------------------------------------
Block#: 3
previous HASH: 00e35824f78df09583e13c1ddcf6e47175c445de
current HASH: 0073c95fda71ba7e2d7ed1f39745b6b3cbc12315

Transactions: 1      Time: 2020-02-04 14:09:14.280881
Sender: 2,  Receiver: 0,  Amount: 0
Merkley Root: 11608d6f51cf2ec61d2bfaeccba50314f4bbea27

NONCE: 530
-------------------------------------------
Block#: 4
previous HASH: 0073c95fda71ba7e2d7ed1f39745b6b3cbc12315
curr